

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/T5_SQL.ipynb)




### Spark NLP documentation and instructions:
https://nlp.johnsnowlabs.com/docs/en/quickstart

### Spark NLP Google T5 Article 	
https://towardsdatascience.com/hands-on-googles-text-to-text-transfer-transformer-t5-with-spark-nlp-6f7db75cecff

### For T5 models:
https://nlp.johnsnowlabs.com/models?q=T5

### You can find details about Spark NLP models here:
https://nlp.johnsnowlabs.com/models



## 1. Colab Setup

In [5]:
# Install PySpark and Spark NLP
! pip install -q pyspark==3.1.2 spark-nlp

     |████████████████████████████████| 212.4 MB 67 kB/s 
     |████████████████████████████████| 140 kB 49.7 MB/s 
     |████████████████████████████████| 198 kB 55.5 MB/s 


# 2. Start the Spark session

In [6]:
import pandas as pd
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
spark = sparknlp.start()

# Choose a sample text

In [7]:
text_list = ["""How many customers have ordered more than 2 items?""","""How many players were with the school or club team La Salle?""","""When the scoring rank was 117, what was the best
finish?""","""When the best finish was T69, how many people came in 2nd?""","""How many wins were there when the money list rank was 183?""","""When did the Metrostars have their first Rookie of the Year winner?""","""What college did the Rookie of the Year from the
Columbus Crew attend?"""]

# T5 for SQL Query generation

In [8]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")
t5 = T5Transformer.pretrained("t5_small_wikiSQL") \
    .setTask("translate English to SQL:") \
    .setInputCols(["documents"]) \
    .setMaxOutputLength(200) \
    .setOutputCol("sql")
pipeline = Pipeline().setStages([documentAssembler, t5])


t5_small_wikiSQL download started this may take some time.
Approximate size to download 249.9 MB
[OK!]


In [12]:
df = spark.createDataFrame(pd.DataFrame(text_list)).toDF("text")

result = pipeline.fit(df).transform(df)

In [16]:

result.select("text","sql.result").show(truncate=False)

+----------------------------------------------------------------------+---------------------------------------------------------------------+
|text                                                                  |result                                                               |
+----------------------------------------------------------------------+---------------------------------------------------------------------+
|How many customers have ordered more than 2 items?                    |[SELECT COUNT Customers FROM table WHERE Orders > 2]                 |
|How many players were with the school or club team La Salle?          |[SELECT COUNT Player FROM table WHERE School/Club Team = La Salle]   |
|When the scoring rank was 117, what was the best
finish?              |[SELECT Best finish FROM table WHERE Scored = 117]                   |
|When the best finish was T69, how many people came in 2nd?            |[SELECT COUNT 2nd FROM table WHERE Best finish = t69]                |